In [7]:
import pandas as pd

conteudo_csv = """municipio, populacao
Niteroi, 515317
Sao Gonçalo, 1091737
Sao Paulo, 12396372
Belo Horzionte, 2530701
Florianopolis, 516524
Santo Andre, 723889
Santoo Andre, 700000
Rio de Janeiro, 6718903
Curitba, 1963726
Brasilia, 3094325"""

with open('input.csv', 'w') as f:
    f.write(conteudo_csv)

df_verificacao = pd.read_csv('input.csv', skipinitialspace=True)
print("Colunas detectadas:", df_verificacao.columns.tolist())

Colunas detectadas: ['municipio', 'populacao']


In [ ]:
import pandas as pd
import requests
import unicodedata
from thefuzz import process

TOKEN = "token apagado por segurança no github"
URL_IBGE = "https://servicodados.ibge.gov.br/api/v1/localidades/municipios"
URL_CORRECAO = "https://mynxlubykylncinttggu.functions.supabase.co/ibge-submit"

def normalizar(t):
    return "".join(c for c in unicodedata.normalize('NFD', str(t)) if unicodedata.category(c) != 'Mn').lower().strip()

print("Consultando API do IBGE...")
municipios_ibge = requests.get(URL_IBGE).json()
nomes_oficiais = [m['nome'] for m in municipios_ibge]
mapa_ibge = {normalizar(m['nome']): m for m in municipios_ibge}

df = pd.read_csv('input.csv', skipinitialspace=True)
resultados = []
ids_processados = set()

print("Processando municípios...")
for _, row in df.iterrows():
    m_input = str(row['municipio']).strip()
    pop_input = int(row['populacao'])

    melhor_match, score = process.extractOne(m_input, nomes_oficiais)

    status = "NAO_ENCONTRADO"
    match = None
    id_ibge = None

    if score > 85:
        match_temp = mapa_ibge.get(normalizar(melhor_match))
        id_temp = match_temp['id']

        if id_temp not in ids_processados:
            ids_processados.add(id_temp)
            match = match_temp
            id_ibge = id_temp
            status = "OK"
        else:
            status = "AMBIGUO"

    if status == "OK":
        resultados.append({
            "municipio_input": m_input,
            "populacao_input": pop_input,
            "municipio_ibge": match['nome'],
            "uf": match['microrregiao']['mesorregiao']['UF']['sigla'],
            "regiao": match['microrregiao']['mesorregiao']['UF']['regiao']['nome'],
            "id_ibge": id_ibge,
            "status": "OK"
        })
    else:
        resultados.append({
            "municipio_input": m_input, "populacao_input": pop_input,
            "municipio_ibge": None, "uf": None, "regiao": None, "id_ibge": None,
            "status": status
        })

df_res = pd.DataFrame(resultados)
df_res.to_csv('resultado.csv', index=False)

df_ok = df_res[df_res['status'] == "OK"]
stats = {
    "total_municipios": len(df_res),
    "total_ok": len(df_ok),
    "total_nao_encontrado": len(df_res[df_res['status'] != "OK"]),
    "total_erro_api": 0,
    "pop_total_ok": int(df_ok['populacao_input'].sum()),
    "medias_por_regiao": df_ok.groupby('regiao')['populacao_input'].mean().to_dict()
}

headers = {
    "Authorization": f"Bearer {TOKEN}",
    "Content-Type": "application/json"
}

print("Enviando resultados para a Nasajon...")
resp = requests.post(URL_CORRECAO, headers=headers, json={"stats": stats})

print("\n--- RESPOSTA FINAL ---")
print(resp.json())

Consultando API do IBGE...
Processando municípios...
Enviando resultados para a Nasajon...

--- RESPOSTA FINAL ---
{'user_id': '0fdce0e1-3063-4efe-be9f-95de3df882eb', 'email': 'alinemeyer00@gmail.com', 'score': 9, 'feedback': 'Excelente! Seu resultado está praticamente idêntico ao gabarito.', 'components': {'integers': {'raw': 4, 'max': 4, 'weighted': 4, 'details': {'total_municipios': {'actual': 10, 'expected': 10, 'ok': True, 'partial': 1}, 'total_ok': {'actual': 9, 'expected': 9, 'ok': True, 'partial': 1}, 'total_nao_encontrado': {'actual': 1, 'expected': 1, 'ok': True, 'partial': 1}, 'total_erro_api': {'actual': 0, 'expected': 0, 'ok': True, 'partial': 1}}}, 'pop_total_ok': {'weighted': 3, 'details': {'actual': 29551494, 'expected': 29551494, 'relDiff': 0, 'partial': 1}}, 'medias_por_regiao': {'raw': 2, 'max': 3, 'weighted': 2, 'details': {'Sudeste': {'actual': 3996153.1666666665, 'expected': 3996153.1666666665, 'ok': True, 'relDiff': 0, 'partial': 1}, 'Sul': {'actual': 1240125, 'e

In [9]:
!pip install thefuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 17.9 MB/s eta 0:00:00
